In [4]:
import pandas as pd
import os


## DADOS terreno

In [5]:
soil_use = pd.read_csv("../Datasets/soil_use_data_training.csv")
terrain_features = pd.read_csv("../Datasets/nasa_terrain_features_training.csv")

## Terrain_features

In [6]:
terrain_features['slope'] = terrain_features['slope'].round(0)
terrain_features['curvature'] = terrain_features['curvature'].round(3)

terrain_features.to_csv("../Datasets/nasa_terrain_features_rounded_training.csv", index=False)

## soil use

In [7]:
percentages = [
    'pct_agri', 'pct_urban', 'pct_natural', 'pct_water', 'pct_wetlands', 'pct_others'
    
]

for percentage in percentages:
    print(percentage)
    soil_use[percentage] = soil_use[percentage].round(0)

soil_use.to_csv("../Datasets/soil_use_data_rounded_training.csv", index=False)

pct_agri
pct_urban
pct_natural
pct_water
pct_wetlands
pct_others
